# Intro


## Goal
**WHAT**: Automatic report generation from Hamilton measurements.  
**WHY**: Speed up the report generation, and avoid human errors (copying data, subjective evaluation, ....)

## Tools
Fast iteration in an agile way.  
Generic approach - different plates setup, prameters, ... all with the same code, no changes needed.  

**Python** programming language.  
**jupyter** notebook is currently used, with some functions divided into small modules.  
**Visual Studio Code** IDE (Integrated Development Environment).  
**Markdown** (*.md) format for generated report (Simple, humanly redable).  

## Input:
 - Worklist file path (*.xls) as used for Hamilton input.
   - Sample name
   - Dilution
   - Viscosity
 - Measurement results file path (*.xls) as output from Hamilton.
 - Parameters; constants in code (file path *.json)
   - CV (Coefficient of variation) threshold
   - Referennce value (1.7954e+10 cp/ml)
   - Dilutions [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0]
   - Decimal digits for output

## Output:
  - Report (*.md, printable to pdf)
    - Could be manually edited
    - Image files
    - Result sheets
  - Estimated size <2kB (current)

## Done
  - Invalid sample:
    - CV >THRESHOLD
    - Only one point
  - Parameters file (*.scv, *.json)
  - Multiple plates (in worklist file)

## TODO:
  - Modules
  - Finalize the report
  - Running modes
    - Python script - automatic run (command line with parameters)
    - GUI; use modules to crete an App (code remains the same, but used from GUI)
  - Tests (unit, integration)
  - checksum (*.sdax); put into report
  - Extensive testing...
  - Automatic print to *.pdf ?

## Conclusion
End to end evaluation time reduction approximately 2h -> 20min per measurement. (thx Felix)


# Generate report  - POC

## Imports

In [1]:
VERBOSE_NOTEBOOK = False
WARNING_DISABLE = True
DEBUG = False

In [2]:
import pandas as pd
import numpy as np
from os import path
import os
import warnings
from scipy.optimize import OptimizeWarning

if WARNING_DISABLE:
    warnings.simplefilter('ignore', RuntimeWarning)
    warnings.simplefilter('ignore', OptimizeWarning)
    warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [3]:
def make_input_paths(input_dir, base_name):
    worklist = path.join(input_dir, base_name + 'worklist-ELISA.xls')
    if not os.path.isfile(worklist):
        raise Exception("Worklist file path is invlaid: {}".format(worklist))

    params = path.join(input_dir, base_name + 'AAV9-ELISA_Parameters.csv')

    return {'worklist': worklist, 'params': params}

def make_output_paths(input_dir, base_name, plate_id):
    results =  path.join(input_dir, base_name + 'calc{}.xlsx'.format(plate_id))
    if not os.path.isfile(results):
        raise Exception("Rewsults file path is invlaid: {}".format(results))
    
    report = path.join(input_dir, 'results_plate_{}'.format(plate_id))
    report = path.join(report, '{}report_plate_{}.md'.format(base_name, plate_id))

    return {'results': results, 'report': report}

In [4]:
WORKING_DIR = './reports/input/'
BASE_NAME = '230426_GN004240-033_-_'

input_files = make_input_paths(WORKING_DIR, BASE_NAME)
WORKLIST_FILE_PATH = input_files['worklist']
PARAMS_FILE_PATH = input_files['params']

DATA_DIR = './data'

## Layouts

In [5]:
from layouthandle import read_plate_layout

PLATE_LAYOUT_ID = 'plate_layout_ident.csv'
PLATE_LAYOUT_NUM = 'plate_layout_num.csv'
PLATE_LAYOUT_DIL_ID = 'plate_layout_dil_id.csv'

g_plate_layout_id = read_plate_layout(path.join(DATA_DIR, PLATE_LAYOUT_ID))
g_plate_layout_num = read_plate_layout(path.join(DATA_DIR, PLATE_LAYOUT_NUM))
g_plate_layout_dil_id = read_plate_layout(path.join(DATA_DIR, PLATE_LAYOUT_DIL_ID))

if VERBOSE_NOTEBOOK:
    display(g_plate_layout_id)
    display(g_plate_layout_num)
    display(g_plate_layout_dil_id)

## Worklist

In [6]:
import worklist as wk
from readdata import read_params

g_wl_raw = wk.read_worklist(WORKLIST_FILE_PATH)
g_valid_plates = wk.check_worklist(g_wl_raw)
g_params = read_params(PARAMS_FILE_PATH)

## Read data

In [7]:
PLATE_ID = 1 # plate id

output_files = make_output_paths(WORKING_DIR, BASE_NAME, PLATE_ID)
RESULT_FILE_PATH = output_files['results']
REPORT_FILE_PATH = output_files['report']
REPORT_DIR = os.path.dirname(os.path.abspath(REPORT_FILE_PATH))

print(REPORT_DIR)

c:\work\report-gen\reports\input\results_plate_1


In [8]:
from readdata import read_concat_data
from readdata import concat_data_with_layouts

g_od = read_concat_data(RESULT_FILE_PATH)
df_all = concat_data_with_layouts(g_od, g_plate_layout_id, g_plate_layout_num, g_plate_layout_dil_id)

if VERBOSE_NOTEBOOK:
    display(g_od)
    display(df_all)

### Dilution to Concentration

Define dilution dataframe. The dataframe is indexed according plate layout, index of refference dataframe corresponds to refference of the `plate_layout_dil`.

In [9]:
# TODO: read reference value from parameters
REF_VAL_MAX = 1.7954e+10
DILUTIONS = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0]

from sample import make_concentration
g_reference_conc = make_concentration(REF_VAL_MAX, DILUTIONS)

if VERBOSE_NOTEBOOK:
    display(g_reference_conc)

## Report generation

In [10]:
from reportmain import report_plate, check_report_crc
from md2pdf.core import md2pdf

EXPORT_TO_PDF = True
CHECK_REPORT_CRC = True
REPORT_PLATES_CRC = [1960526278, 187289463]

def gen_report(valid_plates, worklist, params,
               working_dir, base_name):
    reports = []
    for plate in valid_plates:
        print('Processing plate {} of {}'.format(plate, len(valid_plates)))

        output_files = make_output_paths(working_dir, base_name, plate)
        result_file_path = output_files['results']
        report_file_path = output_files['report']
        report_dir = os.path.dirname(os.path.abspath(report_file_path))
        reports.append(report_plate(plate, worklist, params, 
                    g_plate_layout_id, g_plate_layout_num, g_plate_layout_dil_id,
                    g_reference_conc, result_file_path, report_dir, report_file_path
                    ))
    return reports


reports = gen_report(g_valid_plates, g_wl_raw, g_params, WORKING_DIR, BASE_NAME)
if CHECK_REPORT_CRC:
    for report, crc in zip(reports, REPORT_PLATES_CRC):
        check_report_crc(report, crc)

# if EXPORT_TO_PDF:
#     pdf_file_path = path.join(report_dir,"{}.pdf".format(os.path.basename(report_file_path)))
#     md2pdf(pdf_file_path,
#         md_content=report,
#         md_file_path=None,
#         css_file_path=None,
#         base_url=None)

Processing plate 1 of 2
Processing plate 2 of 2

Report CRC = 1960526278


Report CRC = 187289463



<Figure size 640x480 with 0 Axes>